# Recirculated VAV HVAC System

## Introduction
For our group project we have decided to model a recirculated air variable air volume (VAV) HVAC system with heat recovery, heating coils and adiabatic humidification. The schematic diagram for this system is shown below.

<img src="Model Legend.png" style="margin:auto"/>

## Heat Recovery
We have created a model for the heat recovery component of the system which extracts the heat from the exhaust air and transfers it to the incoming air using a heat exchanger. The three process blocks in the model is the cooling side of the exchanger (***XC***), the heating side of the exchanger (***XH***) and the mixing of the cooled and non-cooled exhaust air (***XM***). ***s*** denotes that the exit flow from XC is saturated vapour, where condensing of the water vapour has occured. The model is shown in the diagram below.

<img src="Heat Exchanger Model.png" style="margin:auto"/>

There are 9 unknown variables in the system:
<p style="text-align: center;">$\theta_s$, $\omega_s$, $\theta_2$, $\theta_4$, $\omega_4$, $\dot{Q}_s$, $\dot{Q}_l$, $\dot{Q}_x$, $\dot{m}_w$ </p>

There are 9 known variables in the system:
<p style="text-align: center;">$\theta_1$, $\omega_1$, $\theta_3$, $\omega_3$, $\gamma$, $\dot{m}_{da}$, $\theta_s^0$, $\omega_s^0$, $f'(\theta_s^0)$</p>

The 3 material characteristics of the the system are:
<p style="text-align: center;">$(UA)_{hx}$, $c_{da}$, $l$

We therefore need 9 linear equations to solve for the number of unknown variables. These are shown below with the unknown values on the left hand side and the known variables on the righthand side:
\begin{equation}
\dot{Q}_x - \dot{Q}_s - \dot{Q}_l = 0  \tag{1.2.0}
\end{equation}
\begin{equation}
(UA)_{hx}\theta_2 - (UA)_{hx}\theta_4 + 2\dot{Q}_s + 2\dot{Q}_l = (UA)_{hx}(\theta_3 - \theta_3) \tag{1.2.1}
\end{equation}
\begin{equation}
\dot{m}_{da}c_{da}\theta_2 - \dot{Q}_x - h_e\dot{m}_{w} = \dot{m}_{da}c_{da}\theta_1 \tag{1.2.2}
\end{equation}
\begin{equation}
\dot{m}_{w} = \dot{m}_{da}(\omega_3 - \omega_4) \tag{1.2.3}
\end{equation}
\begin{equation}
(1 - \gamma)\dot{m}_{da}c_{da}\theta_s + \dot{Q}_s = (1 - \gamma)\dot{m}_{da}c_{da}\theta_3 \tag{1.2.4}
\end{equation}
\begin{equation}
(1 - \gamma)\dot{m}_{da}l\omega_s + \dot{Q}_l = (1 - \gamma)\dot{m}_{da}l\omega_3 \tag{1.2.5}
\end{equation}
\begin{equation}
f'(\theta_s^0)\theta_s - \omega_s = f'(\theta_s^0)\theta_s^0 - \omega_s^0 \tag{1.2.6}
\end{equation}
\begin{equation}
\theta_4 - (1 - \gamma)\theta_s = \gamma\theta_3 \tag{1.2.7}
\end{equation}
\begin{equation}
\omega_4 - (1 - \gamma)\omega_s = \gamma\omega_3 \tag{1.2.8}
\end{equation}
    
The term $h_{e}\dot{m}_{w}$ in equation (1.2.2) can be neglected as $\dot{m}_{w}<<\dot{m}_{da}$. This means we can also neglect equation (1.2.3) as it will have no significant effect on the system. By excluding $\dot{m}_{w}$ this means there are now only 8 unknown variables and therefore only 8 linear equations are required. $(UA)_{hx}$ is the heat transfer coefficient of the heat exchanger and **not** the heat transfer coefficient of the building.

The linear system of equations can be expressed in the matrix form **A** ***x*** = **b** where **A** is an 8 x 8 matrix containing the coefficients of the unknown variables, ***x*** is a 8 x 1 tensor of the unknown variables and **b** is a  8 x 1 tensor of the known values. These are shown below for defined system of linear equations.
    
$ \color{black}{ A= \left(\begin{array}{cc}
0 & 0 & 0 & 0 & 0 & -1 & -1 & 1 \\ 
(UA)_{hx} & 0 & 0 & -(UA)_{hx} & 0 & 2 & 2 & 0 \\ 
\dot{m}_{da}c_{da} & 0 & 0 & 0 & 0 & 0 & 0 & -1 \\ 
0 & (1 - \gamma)\dot{m}_{da}c_{da} & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & (1 - \gamma)\dot{m}_{da}l & 0 & 0 & 1 & 0 & 0 \\
0 & f'(\theta_s^0) & -1 & 0 & 0 & 0 & 0 & 0 \\
0 & -(1 - \gamma) & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & -(1 - \gamma) & 0 & 1 & 0 & 0 & 0
\end{array}\right)
}$

$ \color{black}{ x= \left(\begin{array}{cc}
\theta_2 \\ 
\theta_s \\ 
\omega_s \\ 
\theta_4 \\ 
\omega_4 \\
\dot{Q}_l \\
\dot{Q}_s \\
\dot{Q}_x \\
\end{array}\right)
}$
    
$ \color{black}{ b= \left(\begin{array}{cc}
0 \\ 
(UA)_{hx}(\theta_3 - \theta_4) \\ 
\dot{m}_{da}c_{da}\theta_1 \\ 
(1 - \gamma)\dot{m}_{da}c_{da}\theta_3 \\
(1 - \gamma)\dot{m}_{da}l\omega_3 \\
f'(\theta_s^0)\theta_s^0 - \omega_s^0 \\
\gamma\theta_3 \\
\gamma\omega_3 \\
\end{array}\right)
}$

The code for solving this linear set of equations is shown below which creates a pyschrometric chart with sliders of the input variables to see how the behaviour of the system changes.

## Adiabatic Mixing
In some cases where mixing occurs the resulting mixing point can have a relative humidity greater than 100% which is not physically possible as the water in the air will condense as soon as the air is over saturated. This therefore must be compensated for in the computational model by adding adiabatic mixing, where the latent heat of the air at the mixing point is transferred into sensible heat at a constant specific enthalpy value. This will move the mixing point from the oversaturated region to the saturation curve.

## VAV HVAC Heating and Humidification System